<a href="https://colab.research.google.com/github/prateekbps/DataScience/blob/main/AlexNet_Architecture_Ver_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#60 million param in alexnet vs 60k in lenet5

In [2]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:48<00:00, 24.9MB/s]
100% 1.06G/1.06G [00:48<00:00, 23.4MB/s]


In [4]:
import zipfile
zip_file = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_file.extractall('/content')
zip_file.close()

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.models import Sequential

In [6]:
# Generate all the data in my system
train_ds = keras.utils.image_dataset_from_directory(directory='/content/train', labels='inferred',
                                                    label_mode='int', batch_size=32,
                                                    image_size=(227,227)
                                                    )
test_ds = keras.utils.image_dataset_from_directory(directory='/content/test', labels='inferred',
                                                    label_mode='int', batch_size=32,
                                                    image_size=(227,227)
                                                    )

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [7]:
# Normalize the data
def process(image,label):
  image = tf.cast(image/255.0, tf.float64)
  return image, label

In [8]:
train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [9]:
# CNN Model
model = Sequential()
# Conv 1
model.add(Conv2D(96, kernel_size=(11,11), strides=4, activation='relu', input_shape=(227,227,3)))

# max pooling 1
model.add(MaxPooling2D(pool_size=(3,3), strides=2))

# conv2
model.add(Conv2D(256,kernel_size=(5,5), padding='same', activation='relu'))

# max pooling 2
model.add(MaxPooling2D(pool_size=(3,3), strides=2))

# conv3
model.add(Conv2D(384,kernel_size=(3,3), padding='same', activation='relu'))

# conv4(as no strides)
model.add(Conv2D(384,kernel_size=(3,3), padding='same',activation='relu'))

# conv5(as no strides)
model.add(Conv2D(256,kernel_size=(3,3), padding='same',activation='relu'))

# Maxpooling
model.add(MaxPooling2D(pool_size=(3,3), strides=2))
# flatten
model.add(Flatten())

# DNN 1
model.add(Dense(9216, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
#output
model.add(Dense(1,activation='sigmoid'))

# summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 55, 55, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 27, 27, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 27, 27, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 13, 13, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 13, 13, 384)         │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 13, 13, 384)         │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 13, 13, 256)         │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 9216)                │      84,943,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4096)                │      37,752,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │           4,097 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 143,229,313 (546.38 MB)

 Trainable params: 143,229,313 (546.38 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 75s 106ms/step - accuracy: 0.5035 - loss: 0.8059 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 72s 101ms/step - accuracy: 0.4981 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 63s 101ms/step - accuracy: 0.4971 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 63s 100ms/step - accuracy: 0.4988 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 74s 118ms/step - accuracy: 0.4972 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 99ms/step - accuracy: 0.4965 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 84s 102ms/step - accuracy: 0.4989 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 92s 118ms/step - accuracy: 0.4985 - loss: 0.

In [11]:
# Modified architecture of alexnet

In [12]:
# CNN Model
model = Sequential()
# Conv 1
model.add(Conv2D(32, kernel_size=(3,3), strides=2, padding='same',activation='relu', input_shape=(227,227,3)))
# Batch Normalization to avoid overfitting
model.add(BatchNormalization()) # drop can be applied here
# max pooling 1
model.add(MaxPooling2D(pool_size=(3,3), strides=2))

model.add(Conv2D(32, kernel_size=(3,3), strides=2, padding='same',activation='relu', input_shape=(227,227,3)))
# Batch Normalization to avoid overfitting
model.add(BatchNormalization())
# max pooling 1
model.add(MaxPooling2D(pool_size=(3,3), strides=2))

model.add(Flatten())

# DNN 1
model.add(Dense(9216, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
#output
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_ds, epochs=5, validation_data=test_ds)


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 71s 106ms/step - accuracy: 0.6307 - loss: 2.6567 - val_accuracy: 0.7468 - val_loss: 0.5100
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 68s 89ms/step - accuracy: 0.7622 - loss: 0.5009 - val_accuracy: 0.7918 - val_loss: 0.4963
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 87ms/step - accuracy: 0.7997 - loss: 0.4281 - val_accuracy: 0.8098 - val_loss: 0.4394
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 87ms/step - accuracy: 0.8349 - loss: 0.3710 - val_accuracy: 0.7882 - val_loss: 0.5737
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 56s 90ms/step - accuracy: 0.8596 - loss: 0.3259 - val_accuracy: 0.8002 - val_loss: 0.5745
